# 🏪 Proyecto Aurelion

Este notebook documenta el proceso de **análisis, limpieza y transformación de datos** de la tabla `PRODUCTOS` del proyecto *Aurelion*, utilizando la biblioteca **Pandas** en Python.

El objetivo es preparar un conjunto de datos estructurado, limpio y estandarizado para posteriores procesos de análisis estadístico, modelado o visualización.

## 📊 Descripción General

La tabla `VENTAS` contiene información detallada sobre operaciones comerciales, incluyendo campos de identificación, fechas, montos y categorías de productos.

A lo largo de este notebook se realizarán las siguientes tareas principales:

- Importación de los datos desde un archivo **Excel (.xlsx)**.  
- Creación del dataframe inicial `df_productos_c`.  
- Procesamiento, normalización y renombrado de columnas para generar el dataframe limpio `df_productos`.  
- Aplicación de operaciones básicas de **limpieza**, **transformación** y **estandarización** de los datos.  
- Generación de **estadísticas descriptivas** para comprender la distribución y consistencia de las variables.

## ⚙️ Limpieza y Transformación de los Datos

En esta sección se detallan las operaciones realizadas para asegurar la integridad y coherencia del dataset:

1. **Revisión de valores nulos y duplicados.**  
   - Identificación y tratamiento de valores faltantes.  
   - Eliminación o imputación según el contexto de negocio.

2. **Estandarización de tipos de datos.**  
   - Conversión de fechas, montos y categorías al formato correcto.  
   - Normalización de cadenas y eliminación de espacios o caracteres no deseados.

3. **Renombrado de columnas.**  
   - Aplicación de nombres consistentes y descriptivos conforme a las buenas prácticas de análisis de datos.

4. **Validación del dataframe final.**  
   - Verificación de dimensiones, tipos y contenido.  
   - Comparación con el dataframe original (`df_productos_c`).

#### Importamos librerias instaladas para implementarlas en el código

In [40]:
# Importa paquetes de an?lisis (pandas/numpy) y visualizaci?n (matplotlib/seaborn) con rutas portables
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


## 📥 Carga del Archivo de Datos

En esta sección se realiza la **importación del dataset principal** utilizando una **ruta relativa** y la librería `pathlib.Path`. Esto no solo simplifica la ruta, sino que también garantiza que el código sea **portable** y funcione correctamente en diferentes sistemas operativos (Windows, macOS, Linux), siempre y cuando la estructura de directorios se mantenga consistente.

> 📂 **Ruta del archivo:**  
> El archivo se accede mediante la construcción **`Path('db') / 'productos.xlsx'`**.

La lectura del archivo se efectúa mediante la función `pd.read_excel()` de la biblioteca **pandas**, creando el dataframe inicial `df_productos_c`, el cual servirá como base para los procesos posteriores de limpieza y análisis.

In [41]:
# Lee el catálogo de productos desde la carpeta db usando rutas relativas
path_dataset = Path('db') / 'productos.xlsx'
df_productos_c = pd.read_excel(path_dataset)

# Inspecciona los primeros registros para validar estructura y campos
df_productos_c.head()


,id_producto,nombre_producto,categoria,precio_unitario
0,1,Coca Cola 1.5L,Alimentos,2347
1,2,Pepsi 1.5L,Limpieza,4973
2,3,Sprite 1.5L,Alimentos,4964
3,4,Fanta Naranja 1.5L,Limpieza,2033
4,5,Agua Mineral 500ml,Alimentos,4777


## 🔍 Inspección Inicial del Dataset

En esta etapa se realiza una **inspección exploratoria básica** del DataFrame `df_productos_c` recién cargado, con el objetivo de verificar que los datos se hayan importado correctamente y posean la estructura esperada.

Para ello, se utilizan las funciones:

In [42]:
# Ampl?a la vista inicial a 8 filas para detectar anomal?as tempranas
df_productos_c.head(8)


,id_producto,nombre_producto,categoria,precio_unitario
0,1,Coca Cola 1.5L,Alimentos,2347
1,2,Pepsi 1.5L,Limpieza,4973
2,3,Sprite 1.5L,Alimentos,4964
3,4,Fanta Naranja 1.5L,Limpieza,2033
4,5,Agua Mineral 500ml,Alimentos,4777
5,6,Jugo de Naranja 1L,Limpieza,4170
6,7,Jugo de Manzana 1L,Alimentos,3269
7,8,Energética Nitro 500ml,Limpieza,4218


In [43]:
# Revisa las ?ltimas 4 filas para confirmar consistencia al final del archivo
df_productos_c.tail(4)


,id_producto,nombre_producto,categoria,precio_unitario
96,97,Limpiavidrios 500ml,Alimentos,872
97,98,Desengrasante 500ml,Limpieza,2843
98,99,Esponjas x3,Alimentos,2430
99,100,Trapo de Piso,Limpieza,4854


## 🧠 Exploración de Tipos de Datos y Valores Nulos

En esta etapa se realiza una **revisión estructural del DataFrame** para confirmar que los tipos de datos asignados a cada variable durante la lectura con `pd.read_excel()` sean correctos y coherentes con la naturaleza de la información (por ejemplo, fechas, números, textos, etc.).

Además, se evalúa la **presencia de valores nulos o faltantes**, los cuales podrían requerir tratamiento posterior durante el proceso de limpieza y estandarización.

### 🧩 Procedimiento

In [44]:
# Resume tipos de datos, nulos y memoria del dataframe importado
df_productos_c.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_producto      100 non-null    int64 
 1   nombre_producto  100 non-null    object
 2   categoria        100 non-null    object
 3   precio_unitario  100 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 3.3+ KB


> 💡 **Conclusión:**  
> A partir de la ejecución de la inspección del DataFrame, se observa que la base de datos cuenta con **100 registros** distribuidos en **4 columnas**.  
> Cada columna presenta **100 valores no nulos**, lo que indica que **no existen datos faltantes (NaN)** en el dataset en esta etapa.  
> Los tipos de datos están definidos de la siguiente manera: **campos de identificación y precio** como enteros (`int64`), y **campos descriptivos** (`nombre_producto`, `categoria`) como texto (`object`). Esta estructura inicial es **coherente y está lista** para los procesos de **normalización** (ej. estandarización de texto) y **transformación** (ej. conversión de precios a float).

### 🕵️‍♂️ Detección de Valores Nulos

Se analiza la presencia de valores faltantes para priorizar acciones de limpieza y asegurar la consistencia del dataset.

In [45]:
# Visualiza la máscara booleana donde True marca valores faltantes por celda
df_productos_c.isnull()

,id_producto,nombre_producto,categoria,precio_unitario
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
95,False,False,False,False
96,False,False,False,False
97,False,False,False,False
98,False,False,False,False


In [46]:
# Resume la cantidad de nulos por columna para priorizar limpieza
df_productos_c.isnull().sum()

id_producto        0
nombre_producto    0
categoria          0
precio_unitario    0
dtype: int64

## ⚙️ Normalización y Corrección de Tipos de Datos

Antes de realizar la normalización de la base de datos, es necesario **verificar y corregir los tipos de datos** asignados a las variables y estandarizar los campos de texto. En este caso, la columna `precio_unitario` se convierte al formato `float64` para asegurar precisión decimal. Adicionalmente, se **normalizan** los campos de texto (`nombre_producto` y `categoria`) y se optimiza `categoria` a tipo `categórico` para mejorar la eficiencia del DataFrame.

### 🧩 Fundamento

Corregir los tipos de datos garantiza la **precisión numérica** en cálculos y estadísticas. Si el `precio_unitario` se mantuviera como entero, se perdería información decimal y se podrían producir errores al calcular promedios, totales o al aplicar operaciones aritméticas.

Este paso forma parte del proceso de **limpieza estructural** dentro de la metodología ETL (Extract, Transform, Load):

1. **Extracción:** se importan los datos desde el archivo Excel.  
2. **Transformación estructural:** se ajustan los tipos de datos para asegurar coherencia.  
3. **Normalización:** se reorganiza la base, eliminando redundancias o columnas innecesarias.

### 🧮 Procedimiento

- Se aplican las transformaciones para:

     **Precio:** Asegurar formato `float64` (precisión decimal).

     **Texto:** Limpiar `nombre_producto` y `categoria` (eliminar espacios, unificar mayúsculas/minúsculas).

     **Categóricas:** Convertir `categoria` a tipo `category`.

In [47]:
# 1. Corrección de Tipo de Dato Numérico
# Se convierte 'precio_unitario' a float64 para asegurar precisión decimal.
df_productos_c['precio_unitario'] = df_productos_c['precio_unitario'].astype('float64')

# 2. Normalización de Campos de Texto y Categórico
# Se aplica limpieza de espacios y estandarización a campos de texto.
df_productos_c['nombre_producto'] = (
    df_productos_c['nombre_producto']
    .str.strip()
    .str.title() # Formato Título para nombres de producto
)

# Se aplica limpieza y se convierte a categórico para optimizar memoria y agrupación.
df_productos_c['categoria'] = (
    df_productos_c['categoria']
    .str.strip()
    .str.lower()
    .astype('category')
)

#### 🕵️‍♂️ Verificación de Transformaciones

Se valida que los cambios aplicados hayan surtido efecto tanto en los tipos de datos como en los valores muestreados.

In [48]:
# Verifica los tipos de datos de las columnas normalizadas tras la conversión en la tabla PRODUCTOS
df_productos_c[["id_producto", "nombre_producto", "categoria", "precio_unitario"]].dtypes

id_producto           int64
nombre_producto      object
categoria          category
precio_unitario     float64
dtype: object

In [49]:
# Realiza muestreo de las columnas transformadas para comprobar resultados
df_productos_c[["id_producto", "nombre_producto", "categoria", "precio_unitario"]].head()

,id_producto,nombre_producto,categoria,precio_unitario
0,1,Coca Cola 1.5L,alimentos,2347.0
1,2,Pepsi 1.5L,limpieza,4973.0
2,3,Sprite 1.5L,alimentos,4964.0
3,4,Fanta Naranja 1.5L,limpieza,2033.0
4,5,Agua Mineral 500Ml,alimentos,4777.0


#### 🕵️‍♂️ Control de Calidad: Duplicados

Se evalúa la unicidad de registros, especialmente en la columna `id_producto`, que actúa como identificador clave.

In [50]:
# Evalúa duplicados globales y por id_producto para validar la unicidad clave de productos
cantidad_duplicados = df_productos_c.duplicated().sum()
duplicados_id_producto = df_productos_c['id_producto'].duplicated().sum()

print()
print('--- Registros Duplicados ---')
if cantidad_duplicados > 0:
    print(f'Se encontraron {cantidad_duplicados} filas duplicadas.')
    # Si se encuentran duplicados y se necesita eliminarlos, usar:
    # df_productos_c = df_productos_c.drop_duplicates().copy()
else:
    print('No se encontraron filas duplicadas.')

print()
if duplicados_id_producto > 0:
    print(f'Alerta: se detectaron {duplicados_id_producto} valores repetidos en id_producto (posible clave primaria).')
else:
    print('id_producto es único en df_productos_c.')


--- Registros Duplicados ---
No se encontraron filas duplicadas.

id_producto es único en df_productos_c.


> 💡 **Conclusión:**
> Luego de la verificación y **normalización optimizada** de tipos de datos, se confirma que todas las variables del *dataset* `PRODUCTOS` presentan formatos consistentes y adecuados:
>
> * La columna **`precio_unitario`** fue convertida a **`float64`**, lo cual asegura la **precisión decimal** necesaria para cálculos financieros (ej. promedios y márgenes).
> * Las columnas **`nombre_producto`** y **`id_producto`** mantienen tipos coherentes con su contenido (`object` / `int64`), asegurando la **normalización de texto** (eliminación de espacios y unificación de mayúsculas/minúsculas).
> * La variable **`categoria`** fue convertida al tipo **categórico (`category`)**, lo que facilita las agrupaciones y segmentaciones y genera una **mejora en la eficiencia del procesamiento** y el uso de memoria.
>
> De esta forma, el *dataframe* `df_productos_c` queda estructurado de manera coherente, **garantizando integridad, consistencia, y una ligera mejora en la eficiencia del procesamiento** para las etapas posteriores de transformación y análisis.